In [ ]:
import json
import time
import random
import numpy as np
import os
from datetime import datetime
from openai import OpenAI

client = OpenAI(api_key="YOUR KEY HERE")

# Model settings
model = "o3-2025-04-16"

# Meta-rule configuration
ORDINAL_WORDS = ["first", "second", "third", "fourth", "fifth"]
MAX_CONTEXT_SIZE = 3

# Transformation types with meta-rules - for prompt 2, comment out the description
TRANSFORM_TYPES = {
    'succ': {
        'name': 'Successorship',
        'description': 'The last letter changes to the next letter in the alphabet',
        'rule_number': 1
    },
    'pred': {
        'name': 'Predecessorship', 
        'description': 'The first letter changes to the previous letter in the alphabet',
        'rule_number': 2
    },
    'add_letter': {
        'name': 'Adding a letter',
        'description': 'The next letter in the alphabet after the last letter is added to the end',
        'rule_number': 3
    },
    'remove_redundant': {
        'name': 'Removing redundant character',
        'description': 'If there are duplicate letters, the first duplicate found is removed',
        'rule_number': 4
    },
    'counting': {
        'name': 'Counting',
        'description': 'Count the number of letters in the sequence',
        'rule_number': 5
    }
}

# Meta-rule answers file for alphabet game
ALPHABET_META_ANSWERS_FILE = 'alphabet_meta_answers.json'

def create_system_prompt():
    """Create the system prompt with all rule explanations and meta-rules"""
    prompt = """**Rule Explanations:**
**Rule 1 (Successorship):** The last letter changes to the next letter in the alphabet (e.g., abb -> abc; moose -> moosf; vwxyz -> vwxya)
**Rule 2 (Predecessorship):** The first letter changes to the previous letter in the alphabet (e.g., abb -> zbb; moose -> loose; vwxyz -> uwxyz)  
**Rule 3 (Adding a letter):** The next letter in the alphabet after the last letter is added to the end of the sequence (e.g., abb -> abbc; moose -> moosef; vwxyz -> vwxyza)
**Rule 4 (Removing redundant character):** If there are duplicate letters, the first duplicate found is removed (e.g., abb -> ab; moose -> mose; vwxyz -> vwxyz)
**Rule 5 (Counting):** Count the number of letters in the sequence (e.g., abb -> 3; moose -> 5; vwxyz -> 5)

The base rule of the game is rule 1. Unless otherwise specified, this is the rule you apply.
**Base Rule:** Use **Rule 1** as the default unless instructed otherwise by the meta-rules.
**Meta rule 1**: When you encounter an **ordinal number**, **switch the base rule** to the corresponding numbered rule ("first" apply rule 1, "second" apply rule 2, "third" apply rule 3, "fourth" apply rule 4, "fifth" apply rule 5). Apply this new rule from that point onward until another ordinal number appears, at which point you update the base rule again.

At the end of your response, please provide your final answer in this exact format: Answer: [transformed result]"""
    
    return prompt
#in following comment, prompt 2
'''
def create_system_prompt():
    """Create the system prompt with all rule explanations and meta-rules"""
    prompt = """**Rule Explanations:**
**Rule 1 (Successorship):** e.g., abb -> abc; moose -> moosf; vwxyz -> vwxya
**Rule 2 (Predecessorship):** e.g., abb -> zbb; moose -> loose; vwxyz -> uwxyz
**Rule 3 (Adding letter):** e.g., abb -> abbc; moose -> moosef; vwxyz -> vwxyza
**Rule 4 (Removing redundant character):** e.g., abb -> ab; moose -> mose; vwxwyz -> vwxyz
**Rule 5 (Counting):** e.g., abb -> 3; moose -> 5; vwxyz -> 5

Base Rule: Use Rule 1 as the default unless instructed otherwise by the meta-rules.
**Meta rule 1**: When you encounter an ordinal number, switch the base rule to the corresponding numbered rule ("first" apply rule 1, "second" apply rule 2, "third" apply rule 3, "fourth" apply rule 4, "fifth" apply rule 5). Apply this new rule from that point onward until another ordinal number appears, at which point you update the base rule again.

Keep the reasoning output to a minimum. At the end of your response, please provide your final answer in this EXACT format, do not change the format at all: Answer: [transformed result]"""
    
    return prompt
    '''

def load_alphabet_meta_answers():
    """Load predefined transformations for ordinal words under different rules"""
    try:
        with open(ALPHABET_META_ANSWERS_FILE, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        # Return default transformations if file doesn't exist
        return {
            "first": {
                "succ": "firsu",
                "pred": "eirst",
                "add_letter": "firstu",
                "remove_redundant": "first",
                "counting": "5"
            },
            "second": {
                "succ": "secone",
                "pred": "recond",
                "add_letter": "seconde",
                "remove_redundant": "second",
                "counting": "6"
            },
            "third": {
                "succ": "thire",
                "pred": "shird",
                "add_letter": "thirde",
                "remove_redundant": "third",
                "counting": "5"
            },
            "fourth": {
                "succ": "fourti",
                "pred": "eourth",
                "add_letter": "fourthi",
                "remove_redundant": "fourth",
                "counting": "6"
            },
            "fifth": {
                "succ": "fifti",
                "pred": "eifth",
                "add_letter": "fifthi",
                "remove_redundant": "fith",
                "counting": "5"
            }
        }

def insert_ordinal_word(sequence, probability=0.1):
    """Randomly replace the sequence with 1 ordinal word with given probability"""
    if random.random() > probability:
        return sequence, []
    
    # Select exactly 1 ordinal word to replace the entire sequence
    selected_word = random.choice(ORDINAL_WORDS)
    
    return selected_word, [selected_word]

def determine_active_transform(inserted_words, current_transform):
    """Determine which transformation should be active based on inserted ordinal words"""
    if not inserted_words:
        return current_transform
    
    # Use the last ordinal word to determine the transformation
    transform_mapping = {
        "first": "succ",           # Rule 1
        "second": "pred",          # Rule 2
        "third": "add_letter",     # Rule 3
        "fourth": "remove_redundant", # Rule 4
        "fifth": "counting"        # Rule 5
    }
    
    return transform_mapping.get(inserted_words[-1], current_transform)

def get_transform_result_for_ordinal(word, transform_type, meta_answers):
    """Get the transformation result for an ordinal word under a specific transformation"""
    if word in meta_answers and transform_type in meta_answers[word]:
        return meta_answers[word][transform_type]
    
    # Fallback transformations
    fallback_results = {
        "succ": word + "t",  # Add 't' as successor
        "pred": "a" + word,  # Prepend 'a' as predecessor
        "add_letter": word + word[-1],  # Duplicate last letter
        "remove_redundant": word,  # No change if no duplicates
        "counting": str(len(word))  # Count letters
    }
    
    return fallback_results.get(transform_type, word)

def format_sequence(seq):
    """Format a sequence for display"""
    if isinstance(seq, list):
        return "[" + " ".join(seq) + "]"
    elif isinstance(seq, str):
        return seq
    else:
        return str(seq)

def check_correctness(response, expected_str, transform_type, input_str=None):
    """
    Check if the model's response contains the correct answer by looking for the
    "Answer: " pattern and comparing the provided answer with the expected output.
    
    Args:
        response (str): The full response from the model
        expected_str (str): The expected output string
        transform_type (str): The type of transformation applied
        input_str (str, optional): The original input string
        
    Returns:
        bool: True if the expected output matches the answer, False otherwise
    """
    # Handle empty responses
    if not response or not expected_str:
        return False
    
    # Try to extract the result after "Answer: " if present
    extracted_answer = None
    if "answer:" in response.lower():
        # Split by "Answer:" and take the content after it
        parts = response.lower().split("answer:")
        if len(parts) > 1:
            extracted_answer = parts[1].strip()
            
            # If there are multiple lines after "Answer:", take just the first line
            lines = extracted_answer.split('\n')
            extracted_answer = lines[0].strip()
    
    # If we couldn't extract an answer using the marker, return False
    if not extracted_answer:
        return False
    
    # Clean both strings for comparison
    clean_answer = extracted_answer.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
    clean_expected = expected_str.replace('[', '').replace(']', '').replace('"', '').replace("'", '').strip()
    
    # For "remove_redundant" transformation - must be exact match, not substring
    if transform_type == 'remove_redundant':
        # Remove spaces for comparison
        no_space_answer = clean_answer.replace(' ', '')
        no_space_expected = clean_expected.replace(' ', '')
        
        # Check if the answer is exactly the expected result
        return no_space_answer.lower() == no_space_expected.lower()
    
    # For counting transformation, extract digits
    if transform_type == 'counting':
        # If expected is a number, extract numbers from the answer
        if clean_expected.isdigit():
            import re
            numbers = re.findall(r'\d+', clean_answer)
            return clean_expected in numbers
    
    # For other transformations
    # Remove all spaces for comparison
    no_space_answer = clean_answer.replace(' ', '')
    no_space_expected = clean_expected.replace(' ', '')
    
    # Exact match after normalizing spaces
    if no_space_answer.lower() == no_space_expected.lower():
        return True
        
    # Special case for sequences where spaces matter
    spaced_expected = ' '.join(list(no_space_expected.lower()))
    if clean_answer.lower() == spaced_expected:
        return True
    
    return False

def manage_context(context, new_user_msg, new_assistant_msg, max_size=8, has_ordinal=False):
    """Manage context window size, keeping system message, latest ordinal rule, and recent exchanges"""
    # Add new messages
    context.append({"role": "user", "content": new_user_msg})
    context.append({"role": "assistant", "content": new_assistant_msg})
    
    # If we exceed max size, intelligently manage what to keep
    if len(context) > max_size:
        system_msg = context[0] if context and context[0]["role"] == "system" else None
        
        # Find the most recent message pair that contains an ordinal word
        latest_ordinal_pair = None
        latest_ordinal_index = -1
        
        # Search backwards through user messages for ordinal words
        for i in range(len(context) - 2, 0, -2):  # Step by 2, looking at user messages only
            if i < len(context) and context[i]["role"] == "user":
                user_msg = context[i]["content"].lower()
                if any(word in user_msg for word in ORDINAL_WORDS):
                    latest_ordinal_pair = (context[i], context[i+1])  # user + assistant pair
                    latest_ordinal_index = i
                    break
        
        # If we have ordinal words to preserve
        if latest_ordinal_pair:
            # Keep: system message + latest ordinal pair + most recent exchanges
            other_messages = []
            for i, msg in enumerate(context[1:], 1):  # Skip system message
                # Skip the ordinal pair we're preserving
                if i != latest_ordinal_index and i != latest_ordinal_index + 1:
                    other_messages.append(msg)
            
            # Calculate how many recent messages we can keep
            reserved_slots = 1 + 2  # system + ordinal pair
            available_slots = max_size - reserved_slots
            recent_messages = other_messages[-available_slots:] if available_slots > 0 else []
            
            # Reconstruct context: system + ordinal pair + recent messages
            context = [system_msg] + list(latest_ordinal_pair) + recent_messages
        else:
            # No ordinal words found, use normal context management
            if system_msg:
                recent_messages = context[-(max_size-1):]
                context = [system_msg] + recent_messages
            else:
                context = context[-max_size:]
    
    return context

def main():
    # Load dataset and meta answers
    with open('alphabet_dataset.json', 'r') as f:
        full_dataset = json.load(f)
    
    meta_answers = load_alphabet_meta_answers()
    
    print(f"Loaded dataset with {len(full_dataset)} items")
    
    # Test configuration
    base_transform = 'succ'  # Default transformation (Rule 1)
    current_transform = base_transform
    num_trials = min(400, len(full_dataset))
    
    # Initialize context with system message only
    system_prompt = create_system_prompt()
    context = [{"role": "system", "content": system_prompt}]
    
    # Results tracking
    all_results = []
    correct_answers = 0
    incorrect_answers = 0
    
    print(f"\nTesting alphabet game with meta-rules on {num_trials} items...")
    print(f"Starting with base transformation: Rule {TRANSFORM_TYPES[base_transform]['rule_number']} ({TRANSFORM_TYPES[base_transform]['name']})")
    
    for i in range(num_trials):
        # Select random item
        item = random.choice(full_dataset)
        
        # Maybe insert ordinal words
        original_input = item['input']
        if any(word in str(original_input).lower() for word in ORDINAL_WORDS):
            found_word = next(word for word in ORDINAL_WORDS if word in str(original_input).lower())
            modified_input = original_input  # Keep the original input
            inserted_words = [found_word]    # But mark it as having an ordinal word
        else:
            # Only do random insertion if no natural ordinal found
            modified_input, inserted_words = insert_ordinal_word(original_input)
        
        # Update current transformation based on ordinal words
        if inserted_words:
            current_transform = determine_active_transform(inserted_words, current_transform)
            rule_num = TRANSFORM_TYPES[current_transform]['rule_number']
            print(f"\n🔄 Meta-rule activation! '{inserted_words[-1]}' detected -> Switch to Rule {rule_num} ({TRANSFORM_TYPES[current_transform]['name']})")
        
        # Get expected output
        if inserted_words:
            # For ordinal words, use meta-rule transformations
            expected_output = get_transform_result_for_ordinal(inserted_words[-1], current_transform, meta_answers)
        else:
            # Use regular transformation from dataset
            expected_output = item['transformations'][current_transform]
        
        # Format for display
        input_str = format_sequence(modified_input)
        expected_str = format_sequence(expected_output)
        rule_number = TRANSFORM_TYPES[current_transform]['rule_number']
        
        print(f"\n----- Trial {i+1}/{num_trials} -----")
        print(f"Input: {input_str}")
        if inserted_words:
            print(f"Ordinal words detected: {inserted_words}")
        print(f"Active transformation: Rule {rule_number} ({TRANSFORM_TYPES[current_transform]['name']})")
        print(f"Expected output: {expected_str}")
        
        # Create the transformation task - NO meta-rule explanation in user prompt, only the task
        current_task = f"Transform: {input_str}"
        
        # Create result object
        result = {
            "trial_num": i+1,
            "original_input": original_input,
            "modified_input": modified_input,
            "inserted_words": inserted_words,
            "active_transform": current_transform,
            "rule_number": rule_number,
            "expected_output": expected_output,
            "input_str": input_str,
            "expected_str": expected_str,
            "transform_type": current_transform,
            "context_length": len(context),
            "is_duplicated": item.get('has_duplicates', False),
            "data_type": "word" if isinstance(original_input, str) else "sequence",
            "timestamp": datetime.now().isoformat()
        }
        
        # Call OpenAI API
        try:
            print("Calling API...")
            start_time = time.time()
            
            # Prepare messages
            current_messages = context + [{"role": "user", "content": current_task}]
            
            # Call OpenAI API
            response = client.chat.completions.create(
                model=model,
                messages=current_messages,
                max_completion_tokens = 1000,
                temperature=1
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            response_text = response.choices[0].message.content
            
            print(f"GPT Response: {response_text}")
            
            # Check correctness
            is_correct = check_correctness(response_text, expected_str, current_transform, input_str)

            
            # Update result
            result.update({
                "response": response_text,
                "response_time": response_time,
                "is_correct": is_correct
            })
            
            if is_correct:
                print("✓ Correct!")
                correct_answers += 1
            else:
                print(f"✗ Incorrect. The correct answer is: {expected_str}")
                incorrect_answers += 1
            
            # Update context with sliding window, preserving ordinal words
            context = manage_context(context, current_task, response_text, MAX_CONTEXT_SIZE, bool(inserted_words))
            
        except Exception as e:
            error_msg = str(e)
            print(f"Error: {error_msg}")
            result["error"] = error_msg
            result["is_correct"] = False
            incorrect_answers += 1
        
        all_results.append(result)
        time.sleep(0.5)  # Rate limiting for OpenAI API
    
    # Print final results
    print("\n===== Final Results =====")
    print(f"Total trials: {num_trials}")
    print(f"Correct answers: {correct_answers} ({correct_answers/num_trials*100:.1f}%)")
    print(f"Incorrect answers: {incorrect_answers} ({incorrect_answers/num_trials*100:.1f}%)")
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save JSON
    json_filename = f"{results_dir}/gpto3_alphabet_game_meta_{timestamp}.json"
    with open(json_filename, 'w') as f:
        json.dump(all_results, f, indent=2)
    print(f"Results saved to {json_filename}")
    
    # Save NPZ
    npz_filename = f"{results_dir}/gpto3_alphabet_game_meta_{timestamp}.npz"
    np.savez(
        npz_filename,
        trial_nums=np.array([r["trial_num"] for r in all_results]),
        correctness=np.array([1 if r["is_correct"] else 0 for r in all_results]),
        context_lengths=np.array([r["context_length"] for r in all_results]),
        response_times=np.array([r.get("response_time", 0) for r in all_results]),
        data_types=np.array([1 if r["data_type"] == "word" else 0 for r in all_results]),
        has_duplicates=np.array([1 if r["is_duplicated"] else 0 for r in all_results]),
        has_ordinals=np.array([1 if r["inserted_words"] else 0 for r in all_results]),
        transform_type=current_transform,
        accuracy=correct_answers/num_trials,
        timestamp=timestamp
    )
    print(f"Results also saved to {npz_filename}")
    print("\nDone testing!")

if __name__ == "__main__":
    main()